# Week 17 - Day 1: Black-Scholes and Greeks

## Learning Objectives
- Understand the Black-Scholes option pricing model
- Implement BS formula for European calls and puts
- Calculate and interpret the Greeks (Delta, Gamma, Theta, Vega, Rho)
- Build practical tools for options analysis

## Prerequisites
- Probability and statistics fundamentals
- Basic calculus (partial derivatives)
- Understanding of financial derivatives

In [ ]:
import numpy as np
from scipy import stats
from scipy.optimize import brentq
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

---
## 1. Black-Scholes Model: Theoretical Foundation

### Key Assumptions
1. **Geometric Brownian Motion**: Stock price follows $dS = \mu S dt + \sigma S dW$
2. **No dividends** during the option's life
3. **Constant volatility** $\sigma$ and risk-free rate $r$
4. **No transaction costs** or taxes
5. **Continuous trading** is possible
6. **European options** (exercise only at expiration)

### The Black-Scholes Formula

**Call Option Price:**
$$C = S_0 N(d_1) - K e^{-rT} N(d_2)$$

**Put Option Price:**
$$P = K e^{-rT} N(-d_2) - S_0 N(-d_1)$$

Where:
$$d_1 = \frac{\ln(S_0/K) + (r + \sigma^2/2)T}{\sigma\sqrt{T}}$$
$$d_2 = d_1 - \sigma\sqrt{T}$$

- $S_0$: Current stock price
- $K$: Strike price
- $T$: Time to expiration (years)
- $r$: Risk-free interest rate
- $\sigma$: Volatility (annualized)
- $N(\cdot)$: Cumulative standard normal distribution

In [ ]:
class BlackScholes:
    """
    Black-Scholes Option Pricing Model for European Options.
    
    Parameters:
    -----------
    S : float - Current stock price
    K : float - Strike price
    T : float - Time to expiration (years)
    r : float - Risk-free interest rate
    sigma : float - Volatility (annualized)
    q : float - Dividend yield (default 0)
    """
    
    def __init__(self, S, K, T, r, sigma, q=0):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma
        self.q = q  # Dividend yield
        
        # Pre-calculate d1 and d2
        self._calculate_d1_d2()
    
    def _calculate_d1_d2(self):
        """Calculate d1 and d2 parameters."""
        if self.T <= 0:
            self.d1 = np.inf if self.S > self.K else -np.inf
            self.d2 = self.d1
        else:
            self.d1 = (np.log(self.S / self.K) + 
                      (self.r - self.q + 0.5 * self.sigma**2) * self.T) / \
                      (self.sigma * np.sqrt(self.T))
            self.d2 = self.d1 - self.sigma * np.sqrt(self.T)
    
    def call_price(self):
        """Calculate European call option price."""
        if self.T <= 0:
            return max(self.S - self.K, 0)
        
        N_d1 = stats.norm.cdf(self.d1)
        N_d2 = stats.norm.cdf(self.d2)
        
        call = (self.S * np.exp(-self.q * self.T) * N_d1 - 
                self.K * np.exp(-self.r * self.T) * N_d2)
        return call
    
    def put_price(self):
        """Calculate European put option price."""
        if self.T <= 0:
            return max(self.K - self.S, 0)
        
        N_neg_d1 = stats.norm.cdf(-self.d1)
        N_neg_d2 = stats.norm.cdf(-self.d2)
        
        put = (self.K * np.exp(-self.r * self.T) * N_neg_d2 - 
               self.S * np.exp(-self.q * self.T) * N_neg_d1)
        return put
    
    def price(self, option_type='call'):
        """Calculate option price based on type."""
        if option_type.lower() == 'call':
            return self.call_price()
        elif option_type.lower() == 'put':
            return self.put_price()
        else:
            raise ValueError("option_type must be 'call' or 'put'")


# Example: Price a call and put option
S = 100      # Stock price
K = 100      # Strike price (at-the-money)
T = 0.5      # 6 months to expiration
r = 0.05     # 5% risk-free rate
sigma = 0.20 # 20% volatility

bs = BlackScholes(S, K, T, r, sigma)

print("="*50)
print("BLACK-SCHOLES OPTION PRICING")
print("="*50)
print(f"\nInput Parameters:")
print(f"  Stock Price (S):     ${S:.2f}")
print(f"  Strike Price (K):    ${K:.2f}")
print(f"  Time to Expiry (T):  {T:.2f} years")
print(f"  Risk-free Rate (r):  {r:.2%}")
print(f"  Volatility (σ):      {sigma:.2%}")
print(f"\nIntermediate Values:")
print(f"  d1 = {bs.d1:.6f}")
print(f"  d2 = {bs.d2:.6f}")
print(f"\nOption Prices:")
print(f"  Call Price: ${bs.call_price():.4f}")
print(f"  Put Price:  ${bs.put_price():.4f}")

---
## 2. Put-Call Parity

A fundamental relationship between call and put prices:

$$C - P = S_0 e^{-qT} - K e^{-rT}$$

This relationship provides:
- Arbitrage check
- Alternative pricing method
- Synthetic position creation

In [ ]:
def verify_put_call_parity(S, K, T, r, sigma, q=0):
    """
    Verify put-call parity: C - P = S*e^(-qT) - K*e^(-rT)
    """
    bs = BlackScholes(S, K, T, r, sigma, q)
    
    call = bs.call_price()
    put = bs.put_price()
    
    # Left side of parity
    lhs = call - put
    
    # Right side of parity
    rhs = S * np.exp(-q * T) - K * np.exp(-r * T)
    
    print("PUT-CALL PARITY VERIFICATION")
    print("="*50)
    print(f"Call Price:           ${call:.6f}")
    print(f"Put Price:            ${put:.6f}")
    print(f"\nC - P:                ${lhs:.6f}")
    print(f"S*e^(-qT) - K*e^(-rT): ${rhs:.6f}")
    print(f"\nDifference:           ${abs(lhs - rhs):.10f}")
    print(f"Parity Holds:         {np.isclose(lhs, rhs)}")
    
    return np.isclose(lhs, rhs)

verify_put_call_parity(S, K, T, r, sigma)

---
## 3. The Greeks: Sensitivity Measures

The Greeks measure how option prices change with respect to various factors:

| Greek | Symbol | Measures Sensitivity To | Formula (Call) |
|-------|--------|------------------------|----------------|
| Delta | $\Delta$ | Stock price | $\partial C / \partial S$ |
| Gamma | $\Gamma$ | Delta (2nd order S) | $\partial^2 C / \partial S^2$ |
| Theta | $\Theta$ | Time decay | $\partial C / \partial t$ |
| Vega | $\mathcal{V}$ | Volatility | $\partial C / \partial \sigma$ |
| Rho | $\rho$ | Interest rate | $\partial C / \partial r$ |

### Greek Formulas

**Delta:**
- Call: $\Delta_C = e^{-qT} N(d_1)$
- Put: $\Delta_P = -e^{-qT} N(-d_1)$

**Gamma (same for call and put):**
$$\Gamma = \frac{e^{-qT} n(d_1)}{S \sigma \sqrt{T}}$$

**Theta:**
- Call: $\Theta_C = -\frac{S \sigma e^{-qT} n(d_1)}{2\sqrt{T}} - rKe^{-rT}N(d_2) + qSe^{-qT}N(d_1)$
- Put: $\Theta_P = -\frac{S \sigma e^{-qT} n(d_1)}{2\sqrt{T}} + rKe^{-rT}N(-d_2) - qSe^{-qT}N(-d_1)$

**Vega (same for call and put):**
$$\mathcal{V} = S e^{-qT} \sqrt{T} n(d_1)$$

**Rho:**
- Call: $\rho_C = KT e^{-rT} N(d_2)$
- Put: $\rho_P = -KT e^{-rT} N(-d_2)$

Where $n(x)$ is the standard normal PDF.

In [ ]:
class BlackScholesGreeks(BlackScholes):
    """
    Extended Black-Scholes class with Greeks calculations.
    Inherits from BlackScholes base class.
    """
    
    def delta(self, option_type='call'):
        """
        Delta: Sensitivity to stock price changes.
        - Call delta: 0 to 1 (long stock exposure)
        - Put delta: -1 to 0 (short stock exposure)
        """
        if self.T <= 0:
            if option_type.lower() == 'call':
                return 1.0 if self.S > self.K else 0.0
            else:
                return -1.0 if self.S < self.K else 0.0
        
        discount = np.exp(-self.q * self.T)
        
        if option_type.lower() == 'call':
            return discount * stats.norm.cdf(self.d1)
        else:
            return -discount * stats.norm.cdf(-self.d1)
    
    def gamma(self):
        """
        Gamma: Rate of change of delta.
        Same for calls and puts. Always positive.
        Highest for ATM options near expiration.
        """
        if self.T <= 0:
            return 0.0
        
        discount = np.exp(-self.q * self.T)
        return (discount * stats.norm.pdf(self.d1)) / \
               (self.S * self.sigma * np.sqrt(self.T))
    
    def theta(self, option_type='call', annualized=True):
        """
        Theta: Time decay (typically negative).
        Returns daily theta by default (divide annual by 365).
        """
        if self.T <= 0:
            return 0.0
        
        # Common term
        term1 = -(self.S * self.sigma * np.exp(-self.q * self.T) * 
                  stats.norm.pdf(self.d1)) / (2 * np.sqrt(self.T))
        
        if option_type.lower() == 'call':
            theta = (term1 - 
                    self.r * self.K * np.exp(-self.r * self.T) * stats.norm.cdf(self.d2) +
                    self.q * self.S * np.exp(-self.q * self.T) * stats.norm.cdf(self.d1))
        else:
            theta = (term1 + 
                    self.r * self.K * np.exp(-self.r * self.T) * stats.norm.cdf(-self.d2) -
                    self.q * self.S * np.exp(-self.q * self.T) * stats.norm.cdf(-self.d1))
        
        if not annualized:
            theta = theta / 365  # Daily theta
        
        return theta
    
    def vega(self):
        """
        Vega: Sensitivity to volatility changes.
        Same for calls and puts. Always positive.
        Typically quoted per 1% change in volatility.
        """
        if self.T <= 0:
            return 0.0
        
        vega = (self.S * np.exp(-self.q * self.T) * 
                np.sqrt(self.T) * stats.norm.pdf(self.d1))
        
        return vega / 100  # Per 1% change in volatility
    
    def rho(self, option_type='call'):
        """
        Rho: Sensitivity to interest rate changes.
        Typically quoted per 1% change in rate.
        """
        if self.T <= 0:
            return 0.0
        
        if option_type.lower() == 'call':
            rho = self.K * self.T * np.exp(-self.r * self.T) * stats.norm.cdf(self.d2)
        else:
            rho = -self.K * self.T * np.exp(-self.r * self.T) * stats.norm.cdf(-self.d2)
        
        return rho / 100  # Per 1% change in rate
    
    def all_greeks(self, option_type='call'):
        """Return all Greeks as a dictionary."""
        return {
            'delta': self.delta(option_type),
            'gamma': self.gamma(),
            'theta_annual': self.theta(option_type, annualized=True),
            'theta_daily': self.theta(option_type, annualized=False),
            'vega': self.vega(),
            'rho': self.rho(option_type)
        }


# Calculate Greeks for our example
bsg = BlackScholesGreeks(S, K, T, r, sigma)

print("="*60)
print("THE GREEKS - OPTION SENSITIVITIES")
print("="*60)
print(f"\nOption Parameters: S=${S}, K=${K}, T={T}y, r={r:.1%}, σ={sigma:.1%}")

for opt_type in ['call', 'put']:
    greeks = bsg.all_greeks(opt_type)
    print(f"\n{opt_type.upper()} OPTION (Price: ${bsg.price(opt_type):.4f}):")
    print("-" * 40)
    print(f"  Delta (Δ):        {greeks['delta']:>10.6f}")
    print(f"  Gamma (Γ):        {greeks['gamma']:>10.6f}")
    print(f"  Theta (Θ) annual: {greeks['theta_annual']:>10.4f}")
    print(f"  Theta (Θ) daily:  {greeks['theta_daily']:>10.4f}")
    print(f"  Vega (ν) per 1%:  {greeks['vega']:>10.4f}")
    print(f"  Rho (ρ) per 1%:   {greeks['rho']:>10.4f}")

---
## 4. Visualizing the Greeks

Understanding how Greeks change across different stock prices and time to expiration.

In [ ]:
def plot_greeks_vs_spot(K, T, r, sigma, spot_range=(70, 130), option_type='call'):
    """
    Plot all Greeks as a function of spot price.
    """
    spots = np.linspace(spot_range[0], spot_range[1], 200)
    
    # Calculate Greeks for each spot price
    prices = []
    deltas = []
    gammas = []
    thetas = []
    vegas = []
    rhos = []
    
    for s in spots:
        bsg = BlackScholesGreeks(s, K, T, r, sigma)
        prices.append(bsg.price(option_type))
        deltas.append(bsg.delta(option_type))
        gammas.append(bsg.gamma())
        thetas.append(bsg.theta(option_type, annualized=False))
        vegas.append(bsg.vega())
        rhos.append(bsg.rho(option_type))
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle(f'{option_type.upper()} Option Greeks vs Spot Price\n'
                 f'(K=${K}, T={T}y, r={r:.1%}, σ={sigma:.1%})', fontsize=14)
    
    # Price
    axes[0, 0].plot(spots, prices, 'b-', linewidth=2)
    axes[0, 0].axvline(K, color='r', linestyle='--', alpha=0.5, label='Strike')
    axes[0, 0].set_xlabel('Spot Price')
    axes[0, 0].set_ylabel('Option Price ($)')
    axes[0, 0].set_title('Option Price')
    axes[0, 0].legend()
    
    # Delta
    axes[0, 1].plot(spots, deltas, 'g-', linewidth=2)
    axes[0, 1].axvline(K, color='r', linestyle='--', alpha=0.5)
    axes[0, 1].axhline(0, color='gray', linestyle='-', alpha=0.3)
    axes[0, 1].set_xlabel('Spot Price')
    axes[0, 1].set_ylabel('Delta (Δ)')
    axes[0, 1].set_title('Delta - Stock Price Sensitivity')
    
    # Gamma
    axes[0, 2].plot(spots, gammas, 'purple', linewidth=2)
    axes[0, 2].axvline(K, color='r', linestyle='--', alpha=0.5)
    axes[0, 2].set_xlabel('Spot Price')
    axes[0, 2].set_ylabel('Gamma (Γ)')
    axes[0, 2].set_title('Gamma - Delta Sensitivity')
    
    # Theta
    axes[1, 0].plot(spots, thetas, 'r-', linewidth=2)
    axes[1, 0].axvline(K, color='r', linestyle='--', alpha=0.5)
    axes[1, 0].axhline(0, color='gray', linestyle='-', alpha=0.3)
    axes[1, 0].set_xlabel('Spot Price')
    axes[1, 0].set_ylabel('Theta (Θ) daily')
    axes[1, 0].set_title('Theta - Time Decay')
    
    # Vega
    axes[1, 1].plot(spots, vegas, 'orange', linewidth=2)
    axes[1, 1].axvline(K, color='r', linestyle='--', alpha=0.5)
    axes[1, 1].set_xlabel('Spot Price')
    axes[1, 1].set_ylabel('Vega (ν) per 1%')
    axes[1, 1].set_title('Vega - Volatility Sensitivity')
    
    # Rho
    axes[1, 2].plot(spots, rhos, 'brown', linewidth=2)
    axes[1, 2].axvline(K, color='r', linestyle='--', alpha=0.5)
    axes[1, 2].set_xlabel('Spot Price')
    axes[1, 2].set_ylabel('Rho (ρ) per 1%')
    axes[1, 2].set_title('Rho - Interest Rate Sensitivity')
    
    plt.tight_layout()
    plt.show()

# Plot Greeks for call option
plot_greeks_vs_spot(K=100, T=0.5, r=0.05, sigma=0.20, option_type='call')

In [ ]:
# Plot Greeks for put option
plot_greeks_vs_spot(K=100, T=0.5, r=0.05, sigma=0.20, option_type='put')

---
## 5. Greeks vs Time to Expiration

Understanding how Greeks evolve as expiration approaches is crucial for option traders.

In [ ]:
def plot_greeks_vs_time(S, K, r, sigma, moneyness_levels=[90, 100, 110]):
    """
    Plot Greeks evolution as time to expiration decreases.
    """
    times = np.linspace(0.01, 1.0, 200)  # 1 year down to ~4 days
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Greeks Evolution Over Time (Call Options)', fontsize=14)
    
    colors = ['blue', 'green', 'red']
    
    for K, color in zip(moneyness_levels, colors):
        deltas = []
        gammas = []
        thetas = []
        vegas = []
        
        for t in times:
            bsg = BlackScholesGreeks(S, K, t, r, sigma)
            deltas.append(bsg.delta('call'))
            gammas.append(bsg.gamma())
            thetas.append(bsg.theta('call', annualized=False))
            vegas.append(bsg.vega())
        
        label = f'K=${K} ({"ITM" if K < S else "ATM" if K == S else "OTM"})'
        
        axes[0, 0].plot(times, deltas, color=color, linewidth=2, label=label)
        axes[0, 1].plot(times, gammas, color=color, linewidth=2, label=label)
        axes[1, 0].plot(times, thetas, color=color, linewidth=2, label=label)
        axes[1, 1].plot(times, vegas, color=color, linewidth=2, label=label)
    
    # Delta
    axes[0, 0].set_xlabel('Time to Expiration (Years)')
    axes[0, 0].set_ylabel('Delta')
    axes[0, 0].set_title('Delta vs Time')
    axes[0, 0].legend()
    axes[0, 0].invert_xaxis()
    
    # Gamma
    axes[0, 1].set_xlabel('Time to Expiration (Years)')
    axes[0, 1].set_ylabel('Gamma')
    axes[0, 1].set_title('Gamma vs Time')
    axes[0, 1].legend()
    axes[0, 1].invert_xaxis()
    
    # Theta
    axes[1, 0].set_xlabel('Time to Expiration (Years)')
    axes[1, 0].set_ylabel('Theta (Daily)')
    axes[1, 0].set_title('Theta vs Time')
    axes[1, 0].legend()
    axes[1, 0].invert_xaxis()
    
    # Vega
    axes[1, 1].set_xlabel('Time to Expiration (Years)')
    axes[1, 1].set_ylabel('Vega (per 1%)')
    axes[1, 1].set_title('Vega vs Time')
    axes[1, 1].legend()
    axes[1, 1].invert_xaxis()
    
    plt.tight_layout()
    plt.show()

plot_greeks_vs_time(S=100, K=100, r=0.05, sigma=0.20)

---
## 6. Implied Volatility

Implied Volatility (IV) is the volatility that, when input into the Black-Scholes formula, gives the observed market price.

Since there's no closed-form solution, we use numerical methods (root-finding).

In [ ]:
def implied_volatility(market_price, S, K, T, r, option_type='call', q=0, 
                       bounds=(0.001, 5.0), max_iter=100):
    """
    Calculate implied volatility using Brent's method.
    
    Parameters:
    -----------
    market_price : float - Observed market price of the option
    S, K, T, r : float - Option parameters
    option_type : str - 'call' or 'put'
    q : float - Dividend yield
    bounds : tuple - Search bounds for volatility
    
    Returns:
    --------
    float - Implied volatility
    """
    def objective(sigma):
        bs = BlackScholes(S, K, T, r, sigma, q)
        return bs.price(option_type) - market_price
    
    try:
        iv = brentq(objective, bounds[0], bounds[1], maxiter=max_iter)
        return iv
    except ValueError:
        return np.nan


def implied_volatility_newton(market_price, S, K, T, r, option_type='call', q=0,
                              initial_guess=0.25, tol=1e-8, max_iter=100):
    """
    Calculate IV using Newton-Raphson method (faster convergence).
    Uses Vega as the derivative.
    """
    sigma = initial_guess
    
    for i in range(max_iter):
        bsg = BlackScholesGreeks(S, K, T, r, sigma, q)
        price = bsg.price(option_type)
        vega = bsg.vega() * 100  # Convert back from per 1%
        
        if abs(vega) < 1e-12:
            break
        
        diff = market_price - price
        
        if abs(diff) < tol:
            return sigma
        
        sigma = sigma + diff / vega
        sigma = max(0.001, min(sigma, 5.0))  # Keep within bounds
    
    return sigma


# Example: Calculate IV from a market price
market_call_price = 8.50  # Observed market price
S, K, T, r = 100, 100, 0.5, 0.05

iv_brent = implied_volatility(market_call_price, S, K, T, r, 'call')
iv_newton = implied_volatility_newton(market_call_price, S, K, T, r, 'call')

print("IMPLIED VOLATILITY CALCULATION")
print("="*50)
print(f"Market Call Price: ${market_call_price:.2f}")
print(f"Parameters: S=${S}, K=${K}, T={T}y, r={r:.1%}")
print(f"\nImplied Volatility:")
print(f"  Brent's Method:   {iv_brent:.4%}")
print(f"  Newton-Raphson:   {iv_newton:.4%}")

# Verify
bs_verify = BlackScholes(S, K, T, r, iv_brent)
print(f"\nVerification (using IV={iv_brent:.4%}):")
print(f"  BS Call Price: ${bs_verify.call_price():.4f}")

---
## 7. Option Pricing Surface

Visualizing option prices across different strikes and expirations.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def plot_price_surface(S, r, sigma, option_type='call'):
    """
    3D surface plot of option prices.
    """
    strikes = np.linspace(70, 130, 50)
    times = np.linspace(0.05, 1.0, 50)
    
    K_grid, T_grid = np.meshgrid(strikes, times)
    prices = np.zeros_like(K_grid)
    
    for i in range(len(times)):
        for j in range(len(strikes)):
            bs = BlackScholes(S, strikes[j], times[i], r, sigma)
            prices[i, j] = bs.price(option_type)
    
    fig = plt.figure(figsize=(14, 6))
    
    # 3D Surface
    ax1 = fig.add_subplot(121, projection='3d')
    surf = ax1.plot_surface(K_grid, T_grid, prices, cmap='viridis', alpha=0.8)
    ax1.set_xlabel('Strike Price')
    ax1.set_ylabel('Time to Expiry')
    ax1.set_zlabel('Option Price')
    ax1.set_title(f'{option_type.upper()} Option Price Surface\n(S=${S}, σ={sigma:.0%})')
    fig.colorbar(surf, ax=ax1, shrink=0.5)
    
    # Contour plot
    ax2 = fig.add_subplot(122)
    contour = ax2.contourf(K_grid, T_grid, prices, levels=20, cmap='viridis')
    ax2.axvline(S, color='red', linestyle='--', label=f'Spot=${S}')
    ax2.set_xlabel('Strike Price')
    ax2.set_ylabel('Time to Expiry')
    ax2.set_title(f'{option_type.upper()} Price Contours')
    ax2.legend()
    fig.colorbar(contour, ax=ax2)
    
    plt.tight_layout()
    plt.show()

plot_price_surface(S=100, r=0.05, sigma=0.20, option_type='call')

---
## 8. Practical Example: Option Chain Analysis

Analyzing a complete option chain with Greeks.

In [ ]:
def create_option_chain(S, strikes, T, r, sigma):
    """
    Create an option chain with prices and Greeks.
    """
    chain_data = []
    
    for K in strikes:
        bsg = BlackScholesGreeks(S, K, T, r, sigma)
        
        # Moneyness
        if K < S * 0.98:
            call_money = 'ITM'
            put_money = 'OTM'
        elif K > S * 1.02:
            call_money = 'OTM'
            put_money = 'ITM'
        else:
            call_money = put_money = 'ATM'
        
        chain_data.append({
            'Strike': K,
            # Call side
            'Call_Price': bsg.call_price(),
            'Call_Delta': bsg.delta('call'),
            'Call_Money': call_money,
            # Put side
            'Put_Price': bsg.put_price(),
            'Put_Delta': bsg.delta('put'),
            'Put_Money': put_money,
            # Common Greeks
            'Gamma': bsg.gamma(),
            'Theta_Daily': bsg.theta('call', annualized=False),
            'Vega': bsg.vega()
        })
    
    return chain_data


# Create option chain
S = 100
strikes = [85, 90, 95, 97.5, 100, 102.5, 105, 110, 115]
T = 30/365  # 30 days to expiration
r = 0.05
sigma = 0.25

chain = create_option_chain(S, strikes, T, r, sigma)

print("OPTION CHAIN ANALYSIS")
print(f"Underlying: ${S} | Days to Expiry: 30 | IV: {sigma:.0%}")
print("="*100)
print(f"{'Strike':>8} | {'Call':>8} {'Δ':>7} {'Type':>4} | {'Put':>8} {'Δ':>7} {'Type':>4} | {'Γ':>8} {'Θ/day':>8} {'Vega':>7}")
print("-"*100)

for row in chain:
    print(f"{row['Strike']:>8.1f} | "
          f"${row['Call_Price']:>6.2f} {row['Call_Delta']:>7.4f} {row['Call_Money']:>4} | "
          f"${row['Put_Price']:>6.2f} {row['Put_Delta']:>7.4f} {row['Put_Money']:>4} | "
          f"{row['Gamma']:>8.5f} {row['Theta_Daily']:>8.4f} {row['Vega']:>7.4f}")

---
## 9. Delta Hedging Simulation

Delta hedging is a key application of Greeks in practice. Let's simulate a delta-hedged position.

In [ ]:
def simulate_delta_hedge(S0, K, T, r, sigma, n_steps=252, n_paths=1, rehedge_freq=1):
    """
    Simulate delta hedging of a short call position.
    
    Parameters:
    -----------
    rehedge_freq : int - Rehedge every n steps (1 = daily)
    """
    dt = T / n_steps
    
    # Generate stock price path (GBM)
    np.random.seed(123)
    Z = np.random.standard_normal(n_steps)
    S = np.zeros(n_steps + 1)
    S[0] = S0
    
    for i in range(n_steps):
        S[i+1] = S[i] * np.exp((r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z[i])
    
    # Track hedging P&L
    times = np.linspace(T, 0, n_steps + 1)
    
    # Initial position
    bsg = BlackScholesGreeks(S[0], K, times[0], r, sigma)
    option_value_0 = bsg.call_price()
    delta = bsg.delta('call')
    stock_held = delta  # Shares held for hedging
    cash = option_value_0 - delta * S[0]  # Cash from option premium minus stock purchase
    
    # Track positions
    deltas = [delta]
    hedge_values = [option_value_0]
    option_values = [option_value_0]
    pnl = [0]
    
    for i in range(1, n_steps + 1):
        # Time remaining
        t_remaining = max(times[i], 1e-6)
        
        # Current option value
        bsg = BlackScholesGreeks(S[i], K, t_remaining, r, sigma)
        option_value = bsg.call_price()
        new_delta = bsg.delta('call')
        
        # Hedge portfolio value (before rehedging)
        cash = cash * np.exp(r * dt)  # Cash earns interest
        hedge_portfolio = stock_held * S[i] + cash
        
        # Rehedge if needed
        if i % rehedge_freq == 0:
            delta_change = new_delta - stock_held
            cash = cash - delta_change * S[i]  # Buy/sell stock
            stock_held = new_delta
        
        deltas.append(new_delta)
        hedge_values.append(hedge_portfolio)
        option_values.append(option_value)
        pnl.append(hedge_portfolio - option_value)
    
    # Final P&L
    final_payoff = max(S[-1] - K, 0)
    final_hedge_value = stock_held * S[-1] + cash * np.exp(r * dt)
    
    return {
        'stock_path': S,
        'times': times,
        'deltas': deltas,
        'hedge_values': hedge_values,
        'option_values': option_values,
        'pnl': pnl,
        'final_payoff': final_payoff,
        'final_hedge': final_hedge_value,
        'hedge_error': final_hedge_value - final_payoff
    }


# Run simulation
results = simulate_delta_hedge(S0=100, K=100, T=0.25, r=0.05, sigma=0.20, 
                               n_steps=63, rehedge_freq=1)  # Daily rehedging for 3 months

# Plot results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

days = np.arange(len(results['stock_path']))

# Stock price path
axes[0, 0].plot(days, results['stock_path'], 'b-', linewidth=2)
axes[0, 0].axhline(100, color='r', linestyle='--', alpha=0.5, label='Strike')
axes[0, 0].set_xlabel('Days')
axes[0, 0].set_ylabel('Stock Price')
axes[0, 0].set_title('Stock Price Path')
axes[0, 0].legend()

# Delta evolution
axes[0, 1].plot(days, results['deltas'], 'g-', linewidth=2)
axes[0, 1].set_xlabel('Days')
axes[0, 1].set_ylabel('Delta')
axes[0, 1].set_title('Delta (Hedge Ratio) Over Time')

# Hedge vs Option value
axes[1, 0].plot(days, results['hedge_values'], 'b-', linewidth=2, label='Hedge Portfolio')
axes[1, 0].plot(days, results['option_values'], 'r--', linewidth=2, label='Option Value')
axes[1, 0].set_xlabel('Days')
axes[1, 0].set_ylabel('Value ($)')
axes[1, 0].set_title('Hedge Portfolio vs Option Value')
axes[1, 0].legend()

# Tracking error (P&L)
axes[1, 1].plot(days, results['pnl'], 'purple', linewidth=2)
axes[1, 1].axhline(0, color='gray', linestyle='-', alpha=0.3)
axes[1, 1].set_xlabel('Days')
axes[1, 1].set_ylabel('Tracking Error ($)')
axes[1, 1].set_title('Hedging Error (Hedge - Option)')

plt.tight_layout()
plt.show()

print(f"\nDELTA HEDGING RESULTS:")
print(f"  Final Stock Price:   ${results['stock_path'][-1]:.2f}")
print(f"  Option Payoff:       ${results['final_payoff']:.4f}")
print(f"  Hedge Portfolio:     ${results['final_hedge']:.4f}")
print(f"  Hedging Error:       ${results['hedge_error']:.4f}")

---
## 10. Key Takeaways

### Black-Scholes Model
- Provides analytical solution for European option pricing
- Based on no-arbitrage and risk-neutral valuation
- Assumes constant volatility (major limitation)

### The Greeks
- **Delta**: First-order price sensitivity to underlying; used for hedging
- **Gamma**: Second-order; indicates hedge stability; highest ATM near expiry
- **Theta**: Time decay; typically negative for long positions
- **Vega**: Volatility sensitivity; highest ATM
- **Rho**: Interest rate sensitivity; less significant for short-term options

### Practical Applications
1. **Risk Management**: Greeks quantify portfolio sensitivities
2. **Delta Hedging**: Neutralize directional risk
3. **Implied Volatility**: Market's expectation of future volatility
4. **Position Sizing**: Understanding exposure per contract

### Limitations
- Constant volatility assumption (volatility smile/skew exists)
- No transaction costs (discrete hedging is imperfect)
- Continuous trading assumption
- European options only (American options need other methods)

---
## Exercises

1. **Volatility Impact**: Create a plot showing how call option prices change for different volatility levels (10%, 20%, 30%, 40%) across strikes.

2. **IV Surface**: Given market prices for multiple strikes and expirations, calculate and plot the implied volatility surface.

3. **Gamma Scalping**: Simulate a gamma scalping strategy where you profit from gamma when the market moves while being delta-neutral.

4. **Portfolio Greeks**: Calculate aggregate Greeks for a portfolio of options with different strikes and expirations.

5. **Discrete Hedging**: Compare hedging errors for different rehedging frequencies (daily, weekly, monthly).

In [ ]:
# Exercise 1: Volatility Impact on Option Prices
def plot_volatility_impact():
    """Plot option prices for different volatility levels."""
    strikes = np.linspace(80, 120, 100)
    S, T, r = 100, 0.25, 0.05
    volatilities = [0.10, 0.20, 0.30, 0.40]
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    for sigma in volatilities:
        call_prices = [BlackScholes(S, K, T, r, sigma).call_price() for K in strikes]
        put_prices = [BlackScholes(S, K, T, r, sigma).put_price() for K in strikes]
        
        axes[0].plot(strikes, call_prices, linewidth=2, label=f'σ={sigma:.0%}')
        axes[1].plot(strikes, put_prices, linewidth=2, label=f'σ={sigma:.0%}')
    
    for ax, title in zip(axes, ['Call Options', 'Put Options']):
        ax.axvline(S, color='gray', linestyle='--', alpha=0.5)
        ax.set_xlabel('Strike Price')
        ax.set_ylabel('Option Price ($)')
        ax.set_title(f'{title}: Impact of Volatility')
        ax.legend()
    
    plt.tight_layout()
    plt.show()

plot_volatility_impact()